In [14]:
import pandas as pd
import numpy as np
import re

In [2]:
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0x96 in position 1307: invalid start byte
# encoding='cp1252' solves the issuedf1 = pd.read_csv("result (1).csv"), encoding="cp1252"
df1 = pd.read_csv("./1_test_data/1_test_data_merged_df.csv")
df2 = pd.read_csv("./2_test_data/2_test_data_merged_df.csv")
df3 = pd.read_csv("./3_test_data/3_test_data_merged_df.csv")
# df4 = pd.read_csv("./4_test_data/3_test_data_merged_df.csv")

SUP_EXCL_ARR = ["+ plus super","+super","+ super","+ superannuation", "plus super", "plus superannuation","exclude super","excluding super","excl super","excl of super"];
HOUR_ARR = ["p.h", "p/h", "phr", "hour", "hourly"]

In [143]:
print(len(df1), len(df2), len(df3))

75 187 126


In [3]:
df_list = [df1, df2, df3]

In [11]:
merged_df = pd.concat(df_list)
merged_df = merged_df.reset_index(drop=True)
merged_df = merged_df.drop(columns = 'Unnamed: 0')
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Url                388 non-null    object
 1   Username           144 non-null    object
 2   Title              388 non-null    object
 3   Company            388 non-null    object
 4   Date posted        388 non-null    object
 5   Work type          388 non-null    object
 6   Role description   238 non-null    object
 7   Location           388 non-null    object
 8   Rates              208 non-null    object
 9   Contact name       127 non-null    object
 10  Contact email      115 non-null    object
 11  Contact phone      126 non-null    object
 12  Reference number   9 non-null      object
 13  Current date/time  388 non-null    object
dtypes: object(14)
memory usage: 42.6+ KB


Rates              208/388


In [46]:
# Fill null with empty
# merged_df.fillna("", inplace = True)

In [12]:
# Show duplicates according to Url
merged_df[merged_df['Url'].duplicated() == True].sort_values('Company')

Empty DataFrame
Columns: [Url, Username, Title, Company, Date posted, Work type, Role description, Location, Rates, Contact name, Contact email, Contact phone, Reference number, Current date/time]
Index: []

In [56]:
# re.match('^[0-9]*$', merged_df['Contact phone'])
# (merged_df['Contact phone'].apply(lambda x: re.match('^[0-9.]*$', str(x))) != None).sum()

# 47 out of 126 Rates values contain numbers 
num_df = merged_df[['Company', 'Title', 'Rates']][merged_df['Rates'].apply(lambda x: bool(re.search(r'\d', str(x)))) 
                                                  & merged_df['Rates'].apply(lambda x: bool(re.search('\$', str(x))))] # Find if a value contians numbers and $ sign
print(len(num_df))


175


In [57]:
# reset and drop index
num_df = num_df.reset_index().drop(columns='index')

In [58]:
num_df['Rates'] = num_df['Rates'].str.lower()
num_df = num_df.rename(columns={"Rates": "rates_str"})
num_df

Company                                              Title  \
0          Attribute Group            Technical Business Analyst - Salesforce   
1                  Talenza  Senior Business Analyst - Transformation Delivery   
2                    Aurec                                   Business Analyst   
3    Randstad Technologies                            Senior Business Analyst   
4                  Talenza                  Senior Business Analyst - Banking   
..                     ...                                                ...   
170            The Network                     Data Analyst / Warehouse - NV1   
171      PM-Partners Group              Functional Analyst - BI and Reporting   
172     Ethos BeathChapman  Data Warehouse Consultant / RED - Immediate St...   
173                    PRA                                Azure Data Engineer   
174        Pinpoint Talent                                       Data Analyst   

                               rates_str  
0             up to $900 p/day inc super  
1         $900 - $950 p.d. + plus super   
2               $800 - $950 + super p.d.  
3    $800 - $1050 p.d. + including super  
4          $900 - $960 p.d. + plus super  
..                                   ...  
170                    $700 - $1100 p.d.  
171              daily rate: $850 - $950  
172                  $650 - $725 per day  
173                    $950-$1150p/d doe  
174                  $774.63 + super p/d  

[175 rows x 3 columns]

### Change k to 1000. e.g. $120k, $60k, $1k -> 120000, 60000, 1000

In [59]:
# 'k' in num_df['rates_str'][64]
num_df['rates_str'] = num_df['rates_str'].apply(lambda x: x.replace('k', '000') if 'k' in x else x)

### Get $0,000 format

In [81]:

# Get $00,000 form as well 
num_df['rates'] = num_df['rates_str'].apply(lambda x: re.findall(r'\b\d{1,9}(?:,\d{3})*(?:\.\d+)?(?!\d)', x))
# Remove , 
num_df['rates'] = num_df['rates'].apply(lambda x: re.sub(",", "", str(x)))
# Take numbers again
num_df['rates'] = num_df['rates'].apply(lambda x: re.findall(r'\b\d{1,9}(?:,\d{3})*(?:\.\d+)?(?!\d)', str(x)))

### Replace num_df['rates'][i][0] with num_df['rates'][i][1]

In [82]:
for i in range(len(num_df)):
    if (2 <= len(num_df['rates'][i])) & (float(num_df['rates'][i][0]) < 50):
        if float(num_df['rates'][i][0]) < float(num_df['rates'][i][1]):
            # Replace the first number with the second number
            num_df['rates'][i][0] = float(num_df['rates'][i][1])

In [104]:
# Take the first value
num_df['min_rate'] = num_df['rates'].apply(lambda x: x[0])

# inf is shown becasue it is too big to express with float16
num_df['min_rate'] = num_df['min_rate'].loc[:].astype(np.float32)

In [95]:
num_df['rates_str'][64]

'$900 - $1000 p.d. inc. super'

### Check super, hourly and gst exist

In [96]:

num_df['super'] = 0
num_df['hourly'] = 0
num_df['gst'] = 0

for i in range(len(num_df)):
    for s in SUP_EXCL_ARR:
        if s in num_df['rates_str'][i]:         # if num_df['rates_str'][i].str.contains(s)
            num_df.loc[:, 'super'][i] = 1       # num_df['super'].loc[i] = 1
    
    for h in HOUR_ARR:
        if h in num_df['rates_str'][i]:
            num_df.loc[:, 'hourly'][i] = 1
            
    if 'gst' in num_df['rates_str'][i]:
        num_df.loc[:, 'gst'][i] = 1
        
num_df['super'] = num_df['super'].astype(np.int8) 
num_df['hourly'] = num_df['hourly'].astype(np.int8)
num_df['gst'] = num_df['gst'].astype(np.int8)

C:\Users\GIS\AppData\Local\Temp\ipykernel_15780\2912603717.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_df.loc[:, 'super'][i] = 1       # num_df['super'].loc[i] = 1
C:\Users\GIS\AppData\Local\Temp\ipykernel_15780\2912603717.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_df.loc[:, 'gst'][i] = 1
C:\Users\GIS\AppData\Local\Temp\ipykernel_15780\2912603717.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_df.loc[

In [97]:
num_df.tail()

Company                                              Title  \
170         The Network                     Data Analyst / Warehouse - NV1   
171   PM-Partners Group              Functional Analyst - BI and Reporting   
172  Ethos BeathChapman  Data Warehouse Consultant / RED - Immediate St...   
173                 PRA                                Azure Data Engineer   
174     Pinpoint Talent                                       Data Analyst   

                   rates_str        rates  min_rate  super  hourly  gst  \
170        $700 - $1100 p.d.  [700, 1100]    700.00      0       0    0   
171  daily rate: $850 - $950   [850, 950]    850.00      0       0    0   
172      $650 - $725 per day   [650, 725]    650.00      0       0    0   
173        $950-$1150p/d doe  [950, 1150]    950.00      0       0    0   
174      $774.63 + super p/d     [774.63]    774.63      1       0    0   

     super_added_rate  
170             700.0  
171             850.0  
172             650.0  
173             950.0  
174             855.5

### Super, hourly, GST calculation

In [127]:

num_df['super_added_rate'] = num_df['min_rate'].loc[:]

for i in range(len(num_df)):
    if num_df['hourly'][i] == 1:
        num_df.loc[:, 'super_added_rate'][i] = num_df.loc[:, 'super_added_rate'][i] * 8
    if num_df['super'][i] == 1:
        num_df.loc[:, 'super_added_rate'][i] = num_df.loc[:, 'super_added_rate'][i] * 1.105        
    if num_df['gst'][i] == 1:
        num_df.loc[:, 'super_added_rate'][i] = num_df.loc[:, 'super_added_rate'][i] + num_df.loc[:, 'min_rate'][i] * 0.1

num_df['super_added_rate'] = round(num_df['super_added_rate'], 2)

C:\Users\GIS\AppData\Local\Temp\ipykernel_15780\2175027509.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_df.loc[:, 'super_added_rate'][i] = num_df.loc[:, 'super_added_rate'][i] * 1.105
C:\Users\GIS\AppData\Local\Temp\ipykernel_15780\2175027509.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_df.loc[:, 'super_added_rate'][i] = num_df.loc[:, 'super_added_rate'][i] + num_df.loc[:, 'min_rate'][i] * 0.1
C:\Users\GIS\AppData\Local\Temp\ipykernel_15780\2175027509.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pand

In [133]:
num_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           175 non-null    object 
 1   Title             175 non-null    object 
 2   rates_str         175 non-null    object 
 3   rates             175 non-null    object 
 4   min_rate          175 non-null    float32
 5   super             175 non-null    int8   
 6   hourly            175 non-null    int8   
 7   gst               175 non-null    int8   
 8   super_added_rate  175 non-null    float32
dtypes: float32(2), int8(3), object(4)
memory usage: 7.5+ KB


In [130]:
# num_df[num_df['gst'] == 1]
# num_df[num_df['hourly'] == 1]
num_df.tail()

Company                                              Title  \
170         The Network                     Data Analyst / Warehouse - NV1   
171   PM-Partners Group              Functional Analyst - BI and Reporting   
172  Ethos BeathChapman  Data Warehouse Consultant / RED - Immediate St...   
173                 PRA                                Azure Data Engineer   
174     Pinpoint Talent                                       Data Analyst   

                   rates_str        rates    min_rate  super  hourly  gst  \
170        $700 - $1100 p.d.  [700, 1100]  700.000000      0       0    0   
171  daily rate: $850 - $950   [850, 950]  850.000000      0       0    0   
172      $650 - $725 per day   [650, 725]  650.000000      0       0    0   
173        $950-$1150p/d doe  [950, 1150]  950.000000      0       0    0   
174      $774.63 + super p/d     [774.63]  774.630005      1       0    0   

     super_added_rate  
170        700.000000  
171        850.000000  
172        650.000000  
173        950.000000  
174        855.969971

In [142]:
print(len(num_df[3000 > num_df['super_added_rate']]), len(merged_df))
num_df[3000 > num_df['super_added_rate']].sort_values(by='super_added_rate', ascending = False)

172 388


Company  \
8                                      MBC Recruitment   
125                              Profusion PAC Pty Ltd   
139                                            Talenza   
53                           SustainAbility Consulting   
96                           TRS - IT & Communications   
54                                          GMT People   
73                       Bluefin Resources Pty Limited   
24                                     Mane Consulting   
35                           TRS - IT & Communications   
66                           TRS - IT & Communications   
131                                          Lookahead   
5                                              Talenza   
28                                             Experis   
27                                      Ashdown People   
106                      Bluefin Resources Pty Limited   
119                                TRS - Public Sector   
50                                               Paxus   
22                       Bluefin Resources Pty Limited   
133                                            Talenza   
107                                            Talenza   
14                                               Paxus   
4                                              Talenza   
17                       Bluefin Resources Pty Limited   
70                       Bluefin Resources Pty Limited   
15                       Bluefin Resources Pty Limited   
1                                              Talenza   
99                            AAR Consulting Group P/L   
126          Excelsis Deo - Connecting Talented Humans   
142          Excelsis Deo - Connecting Talented Humans   
12                            AAR Consulting Group P/L   
120                                              Aurec   
143                                TRS - Public Sector   
129                                 Austcorp Executive   
121                                 Austcorp Executive   
25                                             Talenza   
157                                    Mane Consulting   
9                              Northbridge Recruitment   
138                           AAR Consulting Group P/L   
72                           SustainAbility Consulting   
173                                                PRA   
39                                     Mane Consulting   
98                       Bluefin Resources Pty Limited   
78                                  Ethos BeathChapman   
42                              Pathfinder Recruitment   
33                                  Ethos BeathChapman   
6                                              Talenza   
159                Michael Page Information Technology   
11                                            Davidson   
19                                               Aurec   
77                                             Talenza   
0                                      Attribute Group   
13                              Green Light PS Pty Ltd   
64                                             Talenza   
16                           u&u. Recruitment Partners   
152                                       Infinity Pro   
104                                       Infinity Pro   
151                             Hydrogen Group Pty Ltd   
147                           FourQuarters Recruitment   
26                                             Talenza   
74                                             Talenza   
75                           SustainAbility Consulting   
91                       Bluefin Resources Pty Limited   
46                                              Naviro   
89                       Bluefin Resources Pty Limited   
2                                                Aurec   
90                       Bluefin Resources Pty Limited   
37                                             Experis   
44                       Bluefin Resources Pty Limited   
115                      Bluefin Resources Pt

### Compare with data from HubSpot

The number of amounts I got from Python and HubSpot are the same ( 172 values)

In [157]:
num_df1 = num_df[3000 > num_df['super_added_rate']].loc[:].sort_values(by='super_added_rate', ascending = False)
len(num_df1)

172

In [243]:
hubspot_df = pd.read_csv('hubspot-crm-exports-all-deals-2022-10-31.csv')
hubspot_df = hubspot_df[['Deal Name', 'Amount', 'AmountDetail']]
hubspot_df = hubspot_df[0 < hubspot_df['Amount']].sort_values(by='Amount', ascending = False)
hubspot_df['AmountDetail'] = hubspot_df['AmountDetail'].str.lower()
hubspot_df.rename(columns = {'Deal Name' : 'hb_company', 'Amount' : 'hb_amount', 'AmountDetail' : 'hb_detail'}, inplace = True)

len(hubspot_df)

172

In [238]:
num_df2 = num_df1[['Company', 'super_added_rate', 'rates_str']].copy(deep = True)
num_df2.reset_index(drop=True, inplace = True)
num_df2.rename(columns = {'Company' : 'py_company', 'super_added_rate' : 'py_amount', 'rates_str' : 'py_detail'}, inplace = True)
num_df2

py_company  py_amount  \
0                  MBC Recruitment    1215.50   
1            Profusion PAC Pty Ltd    1210.00   
2                          Talenza    1105.00   
3        SustainAbility Consulting    1100.00   
4        TRS - IT & Communications    1100.00   
..                             ...        ...   
167            TRS - Public Sector     497.25   
168            TRS - Public Sector     497.25   
169    Clicks IT Recruitment (NSW)     440.00   
170  Bluefin Resources Pty Limited     400.00   
171            SALT SEARCH PTY LTD     400.00   

                                    py_detail  
0                    $1100 per day plus super  
1         min $1100 pd + gst min 12-36 months  
2                    up to $1000 p.d. + super  
3                 up to $1100 including super  
4    up to $1000 p/d + gst - 6 month contract  
..                                        ...  
167                 $450 - $516 daily + super  
168                 $450 - $516 daily + super  
169                            $55 - $60 p.h.  
170                          $400 - $600 p.d.  
171                          $400 - $600 p.d.  

[172 rows x 3 columns]

In [239]:
compare_df = pd.concat([num_df2, hubspot_df], axis = 1)
compare_df.to_csv('compare_df.csv')

In [247]:
pd.options.display.max_rows = 200

Some amounts show slice differences even though their descriptions and companies are the same. 

In [250]:
print("True:", compare_df['py_amount'].isin(compare_df['hb_amount']).sum())
print("False (below): ", len(compare_df[compare_df['py_amount'].isin(compare_df['hb_amount'])==False]))
compare_df[compare_df['py_amount'].isin(compare_df['hb_amount'])==False]

True: 162
False (below):  10


py_company   py_amount  \
30                              Aurec  983.450012   
31                TRS - Public Sector  972.400024   
77                    Pinpoint Talent  855.969971   
151           Ampersand International  650.010010   
156  Keegan Adams Recruitment Pty Ltd  629.849976   
162   Finite IT Recruitment Solutions  574.599976   
163               TRS - Public Sector  556.919983   
164                   Morgan McKinley  530.400024   
165                    Ashdown People  524.880005   
166                   Mane Consulting  524.880005   

                             py_detail  \
30                       $890 + super    
31                  $880 + super / day   
77                 $774.63 + super p/d   
151      $588.24 - $678.74 p/d + super   
156      $570pd+super ($148,000+super)   
162  hourly rate: $65.00 + super | wfh   
163                 $63.00 p/h + super   
164         $60.0 - $65.0 p.h. + super   
165          $475 - $520 p/day + super   
166      $475 - $520 p.d. + plus super   

                                            hb_company  hb_amount  \
30                  Insub - Aurec - Power BI Developer    983.450   
31   Insub - TRS - Public Sector - Technical Busine...    972.400   
77              Insub - Pinpoint Talent - Data Analyst    855.270   
151  Insub - Ethos BeathChapman - Data Warehouse Co...    650.000   
156  Insub - Keegan Adams Recruitment Pty Ltd - Bus...    629.850   
162  Insub - Finite IT Recruitment Solutions - Data...    574.600   
163  Insub - TRS - Public Sector - Data Analyst (En...    556.920   
164             Insub - Morgan McKinley - Data Manager    530.400   
165  Insub - Ashdown People - Business Analyst | 6 ...    524.875   
166         Insub - Mane Consulting - Business Analyst    524.875   

                             hb_detail  
30                       $890 + super   
31                  $880 + super / day  
77                 $774.63 + super p/d  
151                $650 - $725 per day  
156      $570pd+super ($148,000+super)  
162  hourly rate: $65.00 + super | wfh  
163                 $63.00 p/h + super  
164         $60.0 - $65.0 p.h. + super  
165          $475 - $520 p/day + super  
166      $475 - $520 p.d. + plus super

In [249]:
compare_df

py_company    py_amount  \
0                                      MBC Recruitment  1215.500000   
1                                Profusion PAC Pty Ltd  1210.000000   
2                                              Talenza  1105.000000   
3                            SustainAbility Consulting  1100.000000   
4                            TRS - IT & Communications  1100.000000   
5                                           GMT People  1100.000000   
6                        Bluefin Resources Pty Limited  1049.750000   
7                                      Mane Consulting  1049.000000   
8                            TRS - IT & Communications  1045.000000   
9                            TRS - IT & Communications  1045.000000   
10                                           Lookahead  1000.000000   
11                                             Talenza  1000.000000   
12                                             Experis   994.500000   
13                                      Ashdown People   994.500000   
14                       Bluefin Resources Pty Limited   994.500000   
15                                 TRS - Public Sector   994.500000   
16                                               Paxus   994.500000   
17                       Bluefin Resources Pty Limited   994.500000   
18                                             Talenza   994.500000   
19                                             Talenza   994.500000   
20                                               Paxus   994.500000   
21                                             Talenza   994.500000   
22                       Bluefin Resources Pty Limited   994.500000   
23                       Bluefin Resources Pty Limited   994.500000   
24                       Bluefin Resources Pty Limited   994.500000   
25                                             Talenza   994.500000   
26                            AAR Consulting Group P/L   994.000000   
27           Excelsis Deo - Connecting Talented Humans   990.000000   
28           Excelsis Deo - Connecting Talented Humans   990.000000   
29                            AAR Consulting Group P/L   986.000000   
30                                               Aurec   983.450012   
31                                 TRS - Public Sector   972.400024   
32                                  Austcorp Executive   970.000000   
33                                  Austcorp Executive   970.000000   
34                                             Talenza   950.000000   
35                                     Mane Consulting   950.000000   
36                             Northbridge Recruitment   950.000000   
37                            AAR Consulting Group P/L   950.000000   
38                           SustainAbility Consulting   950.000000   
39                                                 PRA   950.000000   
40                                     Mane Consulting   950.000000   
41                       Bluefin Resources Pty Limited   939.250000   
42                                  Ethos BeathChapman   939.250000   
43                              Pathfinder Recruitment   939.250000   
44                                  Ethos BeathChapman   939.250000   
45                                             Talenza   939.250000   
46                 Michael Page Information Technology   939.250000   
47                                            Davidson   939.250000   
48                                               Aurec   939.250000   
49                                             Talenza   939.250000   
50                                     Attribute Group   900.000000   
51                              Green Light PS Pty Ltd   900.000000   
52                                             Talenza   900.000000   
53                           u&u. Recruitment Partners   900.000000   
54                                        Infinity Pro   900.000000   
55                                        Infinity Pro   900.000000 

In [79]:
# https://stackoverflow.com/questions/63714217/how-can-i-extract-numbers-containing-commas-from-strings-in-python
"""
    \b            word boundary
    \d{1,3}       match 1 to 3 leading digits
    (?:,\d{3})*   followed by zero or more thousands terms
    (?:\.\d+)?    match an optional decimal component
    (?!\d)        assert the "end" of the number by checking for a following non digit
"""

num_df['rates_str'].apply(lambda x: re.findall(r'\b\d{1,9}(?:,\d{3})*(?:\.\d+)?(?!\d)', x))

0            [900]
1       [900, 950]
2       [800, 950]
3      [800, 1050]
4       [900, 960]
          ...     
170    [700, 1100]
171     [850, 950]
172     [650, 725]
173    [950, 1150]
174       [774.63]
Name: rates_str, Length: 175, dtype: object

In [100]:
# test = merged_df[["Company", "Rates"]][merged_df['Rates'].apply(lambda x: bool(re.search('p.d|day|p/d', str(x)))) == True].loc[:]
day_list = ["p.d|pd|p/d|day|daily"]
hour_list = ["p.h", "p/h", "hour", "hourly"]
# Find Rates not containing the values in day_list
test = merged_df[["Company", "Rates"]][merged_df['Rates'].str.lower().str.contains(day_list[0]) == False].loc[:]
print(len(test))
test

21


Company  \
3                 Recruitment Hive   
7          Halcyon Knights Pty Ltd   
9                 Recruitment Hive   
10             TRS - Public Sector   
16                         Talenza   
17                     The Network   
35       SustainAbility Consulting   
44           Hawksworth Consulting   
64         Macquarie Group Limited   
75             Balance Recruitment   
83                Anson McCade Ltd   
92              Ethos BeathChapman   
98                    Talent Stack   
108         2XM Technology Pty Ltd   
117                  Talent Street   
122                 Robert Walters   
131            Skillsearch Pty Ltd   
138                 Quality People   
154  Bluefin Resources Pty Limited   
166                          Aurec   
171        Ampersand International   

                                                 Rates  
3    $110 - $130 p.h. + inc superWeekly/Fortnightly...  
7                                       Market related  
9    $110 - $130 p.h. + inc superWeekly/Fortnightly...  
10                                  $63.00 P/H + Super  
16                                               Super  
17                                             + super  
35                         Up to $1100 including Super  
44                                $100 - $100 per hour  
64                                  12 Month Contract   
75                                       Generous rate  
83                                  Highly competitive  
92                                        $850 + super  
98                                $90 - $120+ per hour  
108                                        Competitive  
117  $90 - $100 per hour, Benefits: AUD 877 ( inclu...  
122                                              super  
131                                $75 - $100 per hour  
138         Contract extensions likely !Top $'s Paid !  
154                                       $825 - $925   
166                                      $890 + Super   
171                           $100 - $140/Hour + Super

In [21]:
merged_df.loc[117]['Rates']

'$90 - $100 per hour, Benefits: AUD 877 ( inclusive'

In [31]:
# merged_df[["Company", "Rates"]][merged_df['Rates'].apply(lambda x: bool(re.search(r'\d', str(x)))) == True].loc[:]